In [148]:
from gensim.models.fasttext import FastText 
from gensim.models.fasttext import load_facebook_model
import textract
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
from scipy.spatial import cKDTree
from gensim.utils import tokenize

#### Betöltjük az előre trainelt modell-t. 

In [8]:
#from gensim.models.wrappers import FastText
model = load_facebook_model('hu.szte.w2v.fasttext.bin')

#### Docx-ből vektort és azért a stringet is kimentjük

In [180]:
def vec_from_doc(file_name, path,  model):
    doc = (
        textract.process(
            f"{path}{file_name}"
        )
        .decode("UTF-8")
        .replace("\n", " ")
    )
    tokens = tokenize(doc)
    return np.nanmean(
        np.array([model.wv.get_vector(word) for word in tokens]), axis=0
    ), doc

In [ ]:
def read_file(path: "Path"):
    doc = textract.process(path).decode("UTF-8").replace("\n", " ")
    tokens = tokenize(doc)
    return tokens

In [ ]:
def preprocess(data: str):
    return np.nanmean(np.array([model.wv.get_vector(word) for word in data]), axis=0)

In [181]:
vec_from_doc(
    "Covid-19 megbeszélés - 2020. március 12_.docx",
    "C:\\Users\\hobot\\Documents\\Prog2\\seeker\\text\\jegyzok\\",
    model,
)

(array([ 0.14983901,  0.00728403, -0.33202073, -0.17792599, -0.13978693,
         0.2790506 , -0.3863321 ,  0.4437692 ,  0.1608899 ,  0.12469662,
         0.11414205,  0.22634624,  0.11562079, -0.0050722 ,  0.10140495,
        -0.02007807,  0.01767357,  0.04711197,  0.13859391,  0.11500996,
        -0.20400965,  0.5022982 ,  0.08677439, -0.02925756, -0.24510187,
        -0.08528614, -0.10353771, -0.0958731 ,  0.25796884,  0.11443552,
        -0.13152488, -0.32832485,  0.03032782, -0.32483742, -0.16829903,
         0.15230997,  0.24770936,  0.2539921 , -0.05800712, -0.2669947 ,
        -0.35128066,  0.22344719,  0.32683665, -0.2501549 ,  0.40868032,
         0.00272824,  0.26762235,  0.0685553 ,  0.00833303, -0.19104186,
        -0.08584382, -0.06062605,  0.1608772 , -0.00586115, -0.13097273,
         0.2106476 , -0.07195537, -0.08077479,  0.25442708,  0.09445832,
         0.15499575, -0.03155251,  0.15755098, -0.12185497,  0.08114625,
         0.07388381,  0.05464004, -0.01579667,  0.0

#### Ez már process rész lenne. Itt készítünk egy dataframe-et címekből (id) és vectorokból (a szövegek model szerinti vektorizált formája)

In [184]:
vectors = []
ids = []
documents = []
for file_name in os.listdir("C:\\Users\\hobot\\Documents\\Prog2\\seeker\\text\\jegyzok"):
    vec, doc = vec_from_doc(file_name,"C:\\Users\\hobot\\Documents\\Prog2\\seeker\\text\\jegyzok\\" ,model)
    ids.append(file_name)
    vectors.append(vec)
    documents.append(doc)

In [185]:
vector_df = pd.DataFrame()
vector_df["ids"] = ids
vector_df["documents"] = documents
vector_df["vectors"] = vectors

#### Ami itt jön az lenne a betáplált jegyzőkönyv. Docx-ben az appra feltöltve. Ezt nevezem példának.

In [189]:
pelda, pelda_doc = vec_from_doc("jegyzo_pelda.docx","C:\\Users\\hobot\\Documents\\Prog2\\seeker\\text\\", model)

#### Building a tree

In [190]:
tree_df = pd.DataFrame(list(map(np.ravel, vectors)))
tree = cKDTree(tree_df)

#### Query

In [191]:
dist, ind = tree.query(pelda, 4)
vector_df.iloc[ind]

,ids,documents,vectors
6,DB Jegyző - 2022.06.13..docx,Múlthét Lugas teaház Geri Nem annyira megy ...,"[0.1503911, -0.024809964, -0.29545856, -0.1886..."
22,DB Jegyzőkönyv - 2020.10.19.docx,"DB Napirend ""Wu Tang Forever"" 26. ülés 2020.1...","[0.15954399, -0.008523128, -0.3089367, -0.1811..."
31,DB jegyzőkönyv 2020.03.02.docx,"DB Napirend ""A párttal, a néppel"" 3. ülés 202...","[0.15086332, -0.016550845, -0.25528255, -0.186..."
19,DB Jegyzőkönyv - 2020.09.07.docx,"DB Napirend 20. ülés 2020.09.07 (offline, ...","[0.15505171, -0.004479873, -0.28971046, -0.195..."


In [80]:
pelda = np.nanmean(
    np.array([model.wv.get_vector(word) for word in pelda_text.split(" ")]), axis=0
)

In [ ]:
from typing import TYPE_CHECKING
from seeker.search.base import BaseModel
#model = load_facebook_model('hu.szte.w2v.fasttext.bin')

if TYPE_CHECKING:
    from pathlib import Path


class NumericModel(BaseModel):
    SEARCH_DIST = "cosine"

    @staticmethod
    def read_file(path: "Path"):
        doc = textract.process(path).decode("UTF-8").replace("\n", " ")
        tokens = tokenize(doc)
        return tokens

    @staticmethod
    def preprocess(data: str):
        return np.nanmean(np.array([model.wv.get_vector(word) for word in data]), axis=0)